In [1]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "Colab Notebooks/image_analysis/brightness"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)


Your working directory was changed to /content/drive/My Drive/Colab Notebooks/image_analysis/brightness

An empty text file was created there. You can also run !pwd to confirm the current working directory.


In [3]:
import PIL
import pandas as pd
from PIL import Image
from PIL import ImageStat
import numpy as np
import pickle

In [ ]:
import os
from os import listdir
from os.path import isfile, join

dir = "/content/drive/MyDrive/Colab Notebooks/airbnb_scraper/pictures/pictures_2st_half_8750_to_full"
pic_paths = [os.path.join(dir, f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f))]

In [ ]:
for i in range(len(pic_paths)):
  if pic_paths[i] == "/content/drive/MyDrive/Colab Notebooks/airbnb_scraper/pictures/pictures_1st_half_8750_to_full/data.pkl": #adjust to folder loc
    pic_paths.pop(i)

In [ ]:
len(pic_paths)

67480

In [67]:
infile = open("/content/drive/MyDrive/Colab Notebooks/airbnb_scraper/pictures/paths_1st_half",'rb')
pic_paths = pickle.load(infile)
infile.close()


In [69]:
pic_paths[1]

'/content/drive/MyDrive/Colab Notebooks/airbnb_scraper/pictures/pictures_1st_half_0_to_8750/23194604_7.png'

In [70]:
pic_number = []
ids = []
full_num = []
for i in range(len(pic_paths)):
  number = pic_paths[i]
  number = number.split("pictures/pictures_1st_half_0_to_8750/")[1] #has to be adjusted to the dir of the pictures
  number = number.split(".png")[0]
  id_s, pic_num = number.split("_")
  id_s = int(id_s)
  pic_num = int(pic_num)
  full_num.append(number)
  pic_number.append(pic_num)
  ids.append(id_s)


In [71]:
pictures_df = pd.DataFrame(
    {"pic_paths": pic_paths,"full_num": full_num, "ids":ids, "pic_number": pic_number}
)

In [ ]:
def brightness_fun( im_file ):
   im = Image.open(im_file)
   stat = ImageStat.Stat(im)
   r,g,b = stat.mean
   return np.sqrt((r**2) + (g**2) + (b**2))

In [72]:
pictures_df

,pic_paths,full_num,ids,pic_number
0,/content/drive/MyDrive/Colab Notebooks/airbnb_...,23194604_6,23194604,6
1,/content/drive/MyDrive/Colab Notebooks/airbnb_...,23194604_7,23194604,7
2,/content/drive/MyDrive/Colab Notebooks/airbnb_...,23194604_8,23194604,8
3,/content/drive/MyDrive/Colab Notebooks/airbnb_...,23194604_9,23194604,9
4,/content/drive/MyDrive/Colab Notebooks/airbnb_...,23194604_10,23194604,10
...,...,...,...,...
68752,/content/drive/MyDrive/Colab Notebooks/airbnb_...,176396_8,176396,8
68753,/content/drive/MyDrive/Colab Notebooks/airbnb_...,176396_9,176396,9
68754,/content/drive/MyDrive/Colab Notebooks/airbnb_...,176396_10,176396,10
68755,/content/drive/MyDrive/Colab Notebooks/airbnb_...,176852_0,176852,0


import numpy as np
a = [1, np.nan, 2, 3]
np.nanmean(a)
2.0
np.nansum(a)
6.0
np.isnan(a)
array([False,  True, False, False], dtype=bool)

In [ ]:
brightness = []
for i in range(len(pictures_df["pic_paths"])):
  try:
    brightness.append(brightness_fun(pictures_df.loc[i,"pic_paths"]))
    print(i)
  except: 
    brightness.append(np.nan)
    continue

import pickle

filename = "/content/drive/MyDrive/Colab Notebooks/image_analysis/brightness/brightness_list_1_simple"
outfile = open(filename,'wb')
pickle.dump(brightness,outfile)
outfile.close()

In [73]:
import pickle
infile = open("/content/drive/MyDrive/Colab Notebooks/image_analysis/brightness/brightness_list_1_simple",'rb')
brightness = pickle.load(infile)
infile.close()

In [ ]:
brightness

In [74]:
pictures_df["brightness"] = brightness

In [50]:
pictures_df[0:5]

,pic_paths,full_num,ids,pic_number,brightness
0,/content/drive/MyDrive/Colab Notebooks/airbnb_...,52262998_5,52262998,5,253.579783
1,/content/drive/MyDrive/Colab Notebooks/airbnb_...,52262998_6,52262998,6,262.407676
2,/content/drive/MyDrive/Colab Notebooks/airbnb_...,52262998_7,52262998,7,258.373697
3,/content/drive/MyDrive/Colab Notebooks/airbnb_...,52265241_0,52265241,0,282.773240
4,/content/drive/MyDrive/Colab Notebooks/airbnb_...,52265241_1,52265241,1,319.961852


In [75]:
max_pic_amount = max(pictures_df["pic_number"]) + 1

In [52]:
max_pic_amount

18

In [76]:
for i in range(max_pic_amount):
  name = "brightness_pic_" + str(i)
  pictures_df[name] = [0] * len(pictures_df)
# adds columns with 0 for each class

In [ ]:
pictures_df

In [77]:
for i in range(len(pictures_df)):
  k = int(pictures_df.loc[i,"pic_number"])
  pictures_df.iloc[i,k + 5] = pictures_df.loc[i,"brightness"]


In [ ]:
pictures_df

In [78]:
better_data_frame = pictures_df.groupby('ids', as_index=False).agg(lambda x: x.tolist())

In [ ]:
better_data_frame[0:5]

In [79]:
final_pic_df = better_data_frame.copy()

In [80]:
final_pic_df = final_pic_df.drop(axis = 1, labels = ["full_num"])

In [ ]:
final_pic_df[0:5]

In [81]:
for i in range(len(final_pic_df)):
  for j in final_pic_df.columns[4:]:
    final_pic_df.loc[i,j] = sum(final_pic_df.loc[i,j])

In [ ]:
final_pic_df

In [82]:
brightness_mean = []
for i in range(len(final_pic_df)):
  brightness_mean.append(np.nanmean(final_pic_df.loc[i,"brightness"]))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until


heck if every value is numeric

In [83]:
for i in range(len(brightness_mean)):
  if not isinstance(brightness_mean[i], np.floating):
    print(i)

In [84]:
final_pic_df.insert(4,"brightness_mean",brightness_mean)

In [85]:
for i in range(len(final_pic_df)):
  final_pic_df.loc[i,"pic_number"] = len(final_pic_df.loc[i,"pic_number"])

In [ ]:
final_pic_df

In [87]:
final_pic_df.to_pickle("brightness_simple_df_1st_half")

In [38]:
infile = open("/content/drive/MyDrive/Colab Notebooks/image_analysis/brightness/brightness_simple_df_1st_half",'rb')
brightness_df_2 = pickle.load(infile)
infile.close()

In [ ]:
any(brightness_df_2["brightness_mean"] == 0)